In [57]:
import pdfplumber
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.util import ngrams
from nltk.corpus import stopwords
import re
import string
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [60]:
def cleanText(page):
    page_text = this_page.extract_text()
    
    clean_text = page_text
    
    # remove punctuation
#     clean_text = re.sub('[%s]' % re.escape(string.punctuation), ' ', clean_text)
    # make lower case
    clean_text = clean_text.lower()
    # remove numbers
    clean_text = re.sub('\w*\d\w*', ' ', clean_text)
      
    # tokenize the text either in individual words or sentences
    page_words = word_tokenize(clean_text)
    page_sentences = sent_tokenize(clean_text) # this doesn't work if punctuation has already been removed, can't clean the punctuation with the same line of code as above once it's been tokenized

#     print(page_sentences)
    
#     print('Num words on page = ', len(page_words))
#     print('Num sentences on page = ', len(page_sentences))

        
    # get counts and remove stop words
#     cv = CountVectorizer(stop_words='english')
#     X = cv.fit_transform(page_sentences)
#     df_cv = pd.DataFrame(X.toarray(), columns=cv.get_feature_names())  
#     df_cv.info()
#     print(df_cv.columns)
#     print(df_cv)

    # get term frequencies and remove stop words
    cv_tfidf = TfidfVectorizer(stop_words='english')
    X_tfidf = cv_tfidf.fit_transform(page_sentences)
    df_tfidf = pd.DataFrame(X_tfidf, columns=cv_tfidf.get_feature_names())
    df_tfidf.info()
    print(df_tfidf.columns)
    print(df_tfidf)        

#     print(page_words)

    
#     twograms = list(ngrams(page_words,2))
#     print(twograms)
    

    

In [61]:
with pdfplumber.open("../StormlightArchiveBooks/The_Way_of_Kings_The_Stormlight_Archive_Book_1.pdf") as pdf:
    this_page = pdf.pages[24]
#     print(this_page.extract_text())
    cleanText(this_page)
    


ValueError: Shape of passed values is (26, 1), indices imply (26, 162)